In [1]:
import pathlib
from scripts.infer.structure import CrossDomainTypes4Py

dataset = CrossDomainTypes4Py(pathlib.Path("/nfs/data/students/bsparks/mdti4py-dataset-pool/cdt4py"))
artifact_root = pathlib.Path("/nfs/home/bsparks/mdti4py/datasets")

2023-09-28 20:24:24.308134: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 20:24:25.255225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import experiments.inferred
entire_gt = experiments.inferred.load_groundtruths(artifact_root, dataset)

import experiments.predictions
trait_form = entire_gt.rename(columns={"adjusted_anno": "trait_gt_form"})
trait_form = trait_form[~trait_form.trait_gt_form.isin(["None", "Any"])]

ubiq_types = experiments.predictions.ubiquitous_types(trait_form)["trait_gt_form"]
common_types = experiments.predictions.common_types(trait_form)["trait_gt_form"]
rare_types = experiments.predictions.rare_types(trait_form)["trait_gt_form"]

/nfs/home/bsparks/mdti4py/datasets/CrossDomainTypes4Py/zwicker-group__numpy-py-pde/extended_ground_truth.csv: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1548/1548 [00:19<00:00, 77.53it/s]
/nfs/home/bsparks/mdti4py/datasets/CrossDomainTypes4Py/zwicker-group__numpy-py-pde/context.csv: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1548/1548 [00:10<00:00, 153.55it/s]


In [3]:
validation_ars = list(map(lambda r: str(dataset.author_repo(r)), dataset.el_validation_set().keys()))
validation_gt = entire_gt[entire_gt["repository"].isin(validation_ars)]

In [4]:
test_ars = list(map(lambda r: str(dataset.author_repo(r)), dataset.el_test_set().keys()))
test_gt = entire_gt[entire_gt["repository"].isin(test_ars)]

In [5]:
del entire_gt
del trait_form

In [6]:
from experiments import pipeline, inferred
type4py_inferred = inferred.load_entire_inferred(
    artifact_root, 
    dataset, 
    tool_name="type4pyN1", 
    task="all"
)
typilus_inferred = inferred.load_entire_inferred(
    artifact_root,
    dataset,
    tool_name="typilusN1",
    task="all"
)
typet5_inferred = inferred.load_entire_inferred(
    artifact_root,
    dataset,
    tool_name="TypeT5TopN1",
    task="all"
)

/nfs/home/bsparks/mdti4py/datasets/CrossDomainTypes4Py/zwicker-group__numpy-py-pde/type4pyN1/all/inferred.csv: 100%|███████████████████████████████████████████████████████████████████████████████████| 1548/1548 [00:11<00:00, 140.71it/s]


Loaded 1538 inference artifacts


/nfs/home/bsparks/mdti4py/datasets/CrossDomainTypes4Py/zwicker-group__numpy-py-pde/typilusN1/all/inferred.csv: 100%|███████████████████████████████████████████████████████████████████████████████████| 1548/1548 [00:10<00:00, 143.87it/s]


Loaded 1541 inference artifacts


/nfs/home/bsparks/mdti4py/datasets/CrossDomainTypes4Py/zwicker-group__numpy-py-pde/TypeT5TopN1/all/inferred.csv: 100%|█████████████████████████████████████████████████████████████████████████████████| 1548/1548 [00:10<00:00, 143.68it/s]


Loaded 1482 inference artifacts


# Load Validation Split

In [8]:
import pandas as pd

In [9]:
type4py_valid_adjusted = pipeline.factory(
    tool="type4pyN1", 
    groundtruth=validation_gt, 
    inferred=type4py_inferred, 
    form="adjusted"
).replace("<MISSING>", pd.NA)
type4py_valid_base = pipeline.factory(
    tool="type4pyN1",
    groundtruth=validation_gt,
    inferred=type4py_inferred,
    form="base"
).replace("<MISSING>", pd.NA)

Initial prediction size: (1166517, 8)
Deriving limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166517/1166517 [00:09<00:00, 124643.69it/s]


Deriving adjusted form from limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166517/1166517 [00:09<00:00, 128175.23it/s]


Size after joining predictions to groundtruth: (826730, 12)


...

Deriving base form from adjusted form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166517/1166517 [00:09<00:00, 125511.78it/s]


(826730, 12)


/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


Reduced to evaluatable: (184404, 12)


In [10]:
typilus_valid_adjusted = pipeline.factory(
    tool="typilusN1", 
    groundtruth=validation_gt, 
    inferred=typilus_inferred, 
    form="adjusted"
).replace("<MISSING>", pd.NA)
typilus_valid_base = pipeline.factory(
    tool="typilusN1",
    groundtruth=validation_gt,
    inferred=typilus_inferred,
    form="base"
).replace("<MISSING>", pd.NA)

Initial prediction size: (1166481, 8)
Deriving limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166481/1166481 [00:09<00:00, 118350.64it/s]


Deriving adjusted form from limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166481/1166481 [00:10<00:00, 115701.01it/s]


Size after joining predictions to groundtruth: (794717, 12)


...

Deriving base form from adjusted form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166481/1166481 [00:09<00:00, 119451.36it/s]


(794717, 12)


/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


Reduced to evaluatable: (179356, 12)


In [11]:
typet5_valid_adjusted = pipeline.factory(
    tool="TypeT5TopN1", 
    groundtruth=validation_gt, 
    inferred=typet5_inferred, 
    form="adjusted"
).replace("<MISSING>", pd.NA)
typet5_valid_base = pipeline.factory(
    tool="TypeT5TopN1",
    groundtruth=validation_gt,
    inferred=typet5_inferred,
    form="base"
).replace("<MISSING>", pd.NA)

Initial prediction size: (1146586, 8)
Deriving limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 222854.61it/s]


Deriving adjusted form from limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 214814.18it/s]


Size after joining predictions to groundtruth: (459890, 12)
Reduced to evaluatable: (166481, 12)


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 216144.09it/s]


Deriving base form from adjusted form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 216981.82it/s]


(459890, 12)
Reduced to evaluatable: (166481, 12)


/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


# Determine performance of models by category

In [12]:
import pandas as pd

In [13]:
(t4py_cat_adj_perf := experiments.predictions.by_category_performance(
    type4py_valid_adjusted, 
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True,
))

observations  predictions  unassigned  matches  \
PARAMETER ubiquitous         54112        52736        1376    37438   
          common             28087        26442        1645    11365   
          rare               30602        28801        1801     6012   
          total             112801       107979        4822    54815   
RETURN    ubiquitous         13787        12926         861     8810   
          common             12313        11206        1107     1500   
          rare               16811        14311        2500       74   
          total              42911        38443        4468    10384   
VARIABLE  ubiquitous         12698        11034        1664     7527   
          common              6895         5426        1469     1310   
          rare                9099         7255        1844      884   
          total              28692        23715        4977     9721   
ALL       ubiquitous         80597        76696        3901    53775   
          common             47295        43074        4221    14175   
          rare               56512        50367        6145     6970   
          total             184404       170137       14267    74920   

                        stracc    relacc  
PARAMETER ubiquitous  0.691861  0.709914  
          common      0.404636  0.429809  
          rare        0.196458  0.208743  
          total       0.485944  0.507645  
RETURN    ubiquitous  0.639008  0.681572  
          common      0.121822  0.133857  
          rare        0.004402  0.005171  
          total       0.241989  0.270114  
VARIABLE  ubiquitous  0.592771  0.682164  
          common      0.189993  0.241430  
          rare        0.097154  0.121847  
          total       0.338805  0.409909  
ALL       ubiquitous  0.667208  0.701145  
          common      0.299715  0.329085  
          rare        0.123337  0.138384  
          total       0.406282  0.440351

In [14]:
(typilus_cat_adj_perf := experiments.predictions.by_category_performance(
    typilus_valid_adjusted, 
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True,
))

observations  predictions  unassigned  matches  \
PARAMETER ubiquitous         52891        51512        1379    36770   
          common             26872        26018         854     5359   
          rare               29852        29273         579     1896   
          total             109615       106803        2812    44025   
RETURN    ubiquitous         13355        12486         869     8111   
          common             11832        11331         501     1391   
          rare               16344        15589         755      402   
          total              41531        39406        2125     9904   
VARIABLE  ubiquitous         12478        11099        1379     6997   
          common              6849         5755        1094      966   
          rare                8883         7319        1564      185   
          total              28210        24173        4037     8148   
ALL       ubiquitous         78724        75097        3627    51878   
          common             45553        43104        2449     7716   
          rare               55079        52181        2898     2483   
          total             179356       170382        8974    62077   

                        stracc    relacc  
PARAMETER ubiquitous  0.695203  0.713814  
          common      0.199427  0.205973  
          rare        0.063513  0.064770  
          total       0.401633  0.412208  
RETURN    ubiquitous  0.607338  0.649608  
          common      0.117563  0.122761  
          rare        0.024596  0.025787  
          total       0.238472  0.251332  
VARIABLE  ubiquitous  0.560747  0.630417  
          common      0.141042  0.167854  
          rare        0.020826  0.025277  
          total       0.288834  0.337070  
ALL       ubiquitous  0.658986  0.690813  
          common      0.169385  0.179009  
          rare        0.045081  0.047584  
          total       0.346111  0.364340

In [15]:
(typet5_cat_adj_perf := experiments.predictions.by_category_performance(
    typet5_valid_adjusted, 
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True,
))

observations  predictions  unassigned  matches  \
PARAMETER ubiquitous         52891        45095        7796    40453   
          common             26872        22638        4234    14912   
          rare               29852        24827        5025    16083   
          total             109615        92560       17055    71448   
RETURN    ubiquitous         13355        11173        2182    10168   
          common             11832        10083        1749     7126   
          rare               16344        13271        3073     8532   
          total              41531        34527        7004    25826   
VARIABLE  ubiquitous          8982         7234        1748     6329   
          common              2522         2093         429     1143   
          rare                3831         3090         741     1777   
          total              15335        12417        2918     9249   
ALL       ubiquitous         75228        63502       11726    56950   
          common             41226        34814        6412    23181   
          rare               50027        41188        8839    26392   
          total             166481       139504       26977   106523   

                        stracc    relacc  
PARAMETER ubiquitous  0.764837  0.897062  
          common      0.554927  0.658715  
          rare        0.538758  0.647803  
          total       0.651809  0.771910  
RETURN    ubiquitous  0.761363  0.910051  
          common      0.602265  0.706734  
          rare        0.522026  0.642906  
          total       0.621849  0.747994  
VARIABLE  ubiquitous  0.704631  0.874896  
          common      0.453212  0.546106  
          rare        0.463848  0.575081  
          total       0.603130  0.744866  
ALL       ubiquitous  0.757032  0.896822  
          common      0.562291  0.665853  
          rare        0.527555  0.640769  
          total       0.639851  0.763584

# Table Creation

In [16]:
# Select total performance only
typilus_cat_adj_perf.index.set_names(["Category", "Scarcity"], inplace=True)
t4py_cat_adj_perf.index.set_names(["Category", "Scarcity"], inplace=True)
typet5_cat_adj_perf.index.set_names(["Category", "Scarcity"], inplace=True)
#pd.concat([
#    typilus_cat_adj_perf.drop(columns="unassigned"),
#    t4py_cat_adj_perf.drop(columns="unassigned"),
#    typet5_cat_adj_perf.drop(columns="unassigned"),
#], keys=["Typilus", "Type4Py", "TypeT5"]).swaplevel()

In [17]:
typilus_cat_adj_perf

observations  predictions  unassigned  matches  \
Category  Scarcity                                                     
PARAMETER ubiquitous         52891        51512        1379    36770   
          common             26872        26018         854     5359   
          rare               29852        29273         579     1896   
          total             109615       106803        2812    44025   
RETURN    ubiquitous         13355        12486         869     8111   
          common             11832        11331         501     1391   
          rare               16344        15589         755      402   
          total              41531        39406        2125     9904   
VARIABLE  ubiquitous         12478        11099        1379     6997   
          common              6849         5755        1094      966   
          rare                8883         7319        1564      185   
          total              28210        24173        4037     8148   
ALL       ubiquitous         78724        75097        3627    51878   
          common             45553        43104        2449     7716   
          rare               55079        52181        2898     2483   
          total             179356       170382        8974    62077   

                        stracc    relacc  
Category  Scarcity                        
PARAMETER ubiquitous  0.695203  0.713814  
          common      0.199427  0.205973  
          rare        0.063513  0.064770  
          total       0.401633  0.412208  
RETURN    ubiquitous  0.607338  0.649608  
          common      0.117563  0.122761  
          rare        0.024596  0.025787  
          total       0.238472  0.251332  
VARIABLE  ubiquitous  0.560747  0.630417  
          common      0.141042  0.167854  
          rare        0.020826  0.025277  
          total       0.288834  0.337070  
ALL       ubiquitous  0.658986  0.690813  
          common      0.169385  0.179009  
          rare        0.045081  0.047584  
          total       0.346111  0.364340

In [18]:
typilus_adj_perf_mask = typilus_cat_adj_perf.index.get_level_values('Scarcity') == 'total'
typilus_adj_perf = typilus_cat_adj_perf.loc[typilus_adj_perf_mask].droplevel(1)


display(typilus_adj_perf)

,observations,predictions,unassigned,matches,stracc,relacc
Category,,,,,,
PARAMETER,109615,106803,2812,44025,0.401633,0.412208
RETURN,41531,39406,2125,9904,0.238472,0.251332
VARIABLE,28210,24173,4037,8148,0.288834,0.337070
ALL,179356,170382,8974,62077,0.346111,0.364340


In [19]:
t4py_adj_perf_mask = t4py_cat_adj_perf.index.get_level_values('Scarcity') == 'total'
t4py_adj_perf = t4py_cat_adj_perf.loc[t4py_adj_perf_mask].droplevel(1)


display(t4py_adj_perf)

,observations,predictions,unassigned,matches,stracc,relacc
Category,,,,,,
PARAMETER,112801,107979,4822,54815,0.485944,0.507645
RETURN,42911,38443,4468,10384,0.241989,0.270114
VARIABLE,28692,23715,4977,9721,0.338805,0.409909
ALL,184404,170137,14267,74920,0.406282,0.440351


In [20]:
tt5_adj_perf_mask = typet5_cat_adj_perf.index.get_level_values('Scarcity') == 'total'
tt5_adj_perf = typet5_cat_adj_perf.loc[tt5_adj_perf_mask].droplevel(1)


display(tt5_adj_perf)

,observations,predictions,unassigned,matches,stracc,relacc
Category,,,,,,
PARAMETER,109615,92560,17055,71448,0.651809,0.771910
RETURN,41531,34527,7004,25826,0.621849,0.747994
VARIABLE,15335,12417,2918,9249,0.603130,0.744866
ALL,166481,139504,26977,106523,0.639851,0.763584


In [21]:
model_by_category = pd.concat(
    [typilus_adj_perf.drop(columns=["unassigned"]), t4py_adj_perf.drop(columns=["unassigned"]), tt5_adj_perf.drop(columns=["unassigned"])],
    keys=["Typilus", "Type4Py", "TypeT5"]
)
model_by_category.index.set_names(["Model", "Category"], inplace=True)
display(model_by_category)

observations  predictions  matches    stracc    relacc
Model   Category                                                         
Typilus PARAMETER        109615       106803    44025  0.401633  0.412208
        RETURN            41531        39406     9904  0.238472  0.251332
        VARIABLE          28210        24173     8148  0.288834  0.337070
        ALL              179356       170382    62077  0.346111  0.364340
Type4Py PARAMETER        112801       107979    54815  0.485944  0.507645
        RETURN            42911        38443    10384  0.241989  0.270114
        VARIABLE          28692        23715     9721  0.338805  0.409909
        ALL              184404       170137    74920  0.406282  0.440351
TypeT5  PARAMETER        109615        92560    71448  0.651809  0.771910
        RETURN            41531        34527    25826  0.621849  0.747994
        VARIABLE          15335        12417     9249  0.603130  0.744866
        ALL              166481       139504   106523  0.639851  0.763584

In [22]:
category_by_model = pd.concat([g for _, g in model_by_category.swaplevel().groupby("Category", sort=False)])
display(category_by_model)

observations  predictions  matches    stracc    relacc
Category  Model                                                          
PARAMETER Typilus        109615       106803    44025  0.401633  0.412208
          Type4Py        112801       107979    54815  0.485944  0.507645
          TypeT5         109615        92560    71448  0.651809  0.771910
RETURN    Typilus         41531        39406     9904  0.238472  0.251332
          Type4Py         42911        38443    10384  0.241989  0.270114
          TypeT5          41531        34527    25826  0.621849  0.747994
VARIABLE  Typilus         28210        24173     8148  0.288834  0.337070
          Type4Py         28692        23715     9721  0.338805  0.409909
          TypeT5          15335        12417     9249  0.603130  0.744866
ALL       Typilus        179356       170382    62077  0.346111  0.364340
          Type4Py        184404       170137    74920  0.406282  0.440351
          TypeT5         166481       139504   106523  0.639851  0.763584

In [23]:
print(category_by_model.to_latex(
    float_format="{:.2f}".format,
))

\begin{tabular}{llrrrrr}
\toprule
 &  & observations & predictions & matches & stracc & relacc \\
Category & Model &  &  &  &  &  \\
\midrule
\multirow[t]{3}{*}{PARAMETER} & Typilus & 109615 & 106803 & 44025 & 0.40 & 0.41 \\
 & Type4Py & 112801 & 107979 & 54815 & 0.49 & 0.51 \\
 & TypeT5 & 109615 & 92560 & 71448 & 0.65 & 0.77 \\
\cline{1-7}
\multirow[t]{3}{*}{RETURN} & Typilus & 41531 & 39406 & 9904 & 0.24 & 0.25 \\
 & Type4Py & 42911 & 38443 & 10384 & 0.24 & 0.27 \\
 & TypeT5 & 41531 & 34527 & 25826 & 0.62 & 0.75 \\
\cline{1-7}
\multirow[t]{3}{*}{VARIABLE} & Typilus & 28210 & 24173 & 8148 & 0.29 & 0.34 \\
 & Type4Py & 28692 & 23715 & 9721 & 0.34 & 0.41 \\
 & TypeT5 & 15335 & 12417 & 9249 & 0.60 & 0.74 \\
\cline{1-7}
\multirow[t]{3}{*}{ALL} & Typilus & 179356 & 170382 & 62077 & 0.35 & 0.36 \\
 & Type4Py & 184404 & 170137 & 74920 & 0.41 & 0.44 \\
 & TypeT5 & 166481 & 139504 & 106523 & 0.64 & 0.76 \\
\cline{1-7}
\bottomrule
\end{tabular}



# Load Test Split

In [24]:
type4py_test_adjusted = pipeline.factory(
    tool="type4pyN1", 
    groundtruth=test_gt, 
    inferred=type4py_inferred, 
    form="adjusted"
).replace("<MISSING>", pd.NA)
type4py_test_base = pipeline.factory(
    tool="type4pyN1",
    groundtruth=test_gt,
    inferred=type4py_inferred,
    form="base"
).replace("<MISSING>", pd.NA)

Initial prediction size: (1166517, 8)
Deriving limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166517/1166517 [00:09<00:00, 124927.82it/s]


Deriving adjusted form from limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166517/1166517 [00:08<00:00, 132115.93it/s]


Size after joining predictions to groundtruth: (223022, 12)
Reduced to evaluatable: (41562, 12)


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166517/1166517 [00:09<00:00, 129219.80it/s]


Deriving base form from adjusted form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166517/1166517 [00:09<00:00, 128579.45it/s]


(223022, 12)
Reduced to evaluatable: (41562, 12)


/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


In [25]:
display(experiments.predictions.performance(
    type4py_test_adjusted,
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18871,17879,992,12929,0.685125,0.723139
common,10583,9717,866,3308,0.312577,0.340434
rare,12108,10680,1428,1314,0.108523,0.123034
total,41562,38276,3286,17551,0.422285,0.458538


In [26]:
typilus_test_adjusted = pipeline.factory(
    tool="typilusN1", 
    groundtruth=test_gt, 
    inferred=typilus_inferred, 
    form="adjusted"
).replace("<MISSING>", pd.NA)
typilus_test_base = pipeline.factory(
    tool="typilusN1",
    groundtruth=test_gt,
    inferred=typilus_inferred,
    form="base"
).replace("<MISSING>", pd.NA)

Initial prediction size: (1166481, 8)
Deriving limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166481/1166481 [00:09<00:00, 117872.87it/s]


Deriving adjusted form from limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166481/1166481 [00:10<00:00, 115204.29it/s]


Size after joining predictions to groundtruth: (213515, 12)
Reduced to evaluatable: (40630, 12)


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166481/1166481 [00:09<00:00, 117614.38it/s]


Deriving base form from adjusted form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1166481/1166481 [00:09<00:00, 117991.26it/s]


(213515, 12)
Reduced to evaluatable: (40630, 12)


/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


In [27]:
display(experiments.predictions.performance(
    typilus_test_adjusted,
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18479,17559,920,12161,0.658098,0.692579
common,10378,9933,445,1704,0.164193,0.171549
rare,11773,11026,747,259,0.021999,0.023490
total,40630,38518,2112,14124,0.347625,0.366686


In [28]:
typet5_test_adjusted = pipeline.factory(
    tool="TypeT5TopN1", 
    groundtruth=test_gt, 
    inferred=typet5_inferred, 
    form="adjusted"
).replace("<MISSING>", pd.NA)
typet5_test_base = pipeline.factory(
    tool="TypeT5TopN1",
    groundtruth=test_gt,
    inferred=typet5_inferred,
    form="base"
).replace("<MISSING>", pd.NA)

Initial prediction size: (1146586, 8)
Deriving limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 218352.11it/s]


Deriving adjusted form from limited form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 212738.32it/s]


Size after joining predictions to groundtruth: (123095, 12)
Reduced to evaluatable: (38413, 12)


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 212642.05it/s]


Deriving base form from adjusted form


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1146586/1146586 [00:05<00:00, 212434.83it/s]


(123095, 12)
Reduced to evaluatable: (38413, 12)


/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


In [87]:
display(typet5_test_adj_perf := experiments.predictions.performance(
    typet5_test_adjusted,
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,17903,15045,2858,13469,0.752332,0.895248
common,9623,8385,1238,6067,0.630469,0.723554
rare,10887,9472,1415,6259,0.574906,0.660790
total,38413,32902,5511,25795,0.671517,0.783995


In [88]:
display(typet5_test_base_perf := experiments.predictions.performance(
    typet5_test_base,
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,17903,15045,2858,13720,0.766352,0.911931
common,9623,8385,1238,6968,0.724099,0.831008
rare,10887,9472,1415,7061,0.648572,0.745460
total,38413,32902,5511,27749,0.722386,0.843383


# Create Prediction Masks

In [70]:
adjusted_fill_me_in = test_gt.drop(columns=[
    'raw_anno',
    'depth_limited_anno',
    'adjusted_anno',
    'base_anno',
    'context_category',
    'nested'
])
display(adjusted_fill_me_in.columns)
display(adjusted_fill_me_in.shape)

key = ["repository", "file", "category", "qname_ssa", "qname"]

Index(['file', 'category', 'qname', 'qname_ssa', 'repository'], dtype='object')

(229080, 5)

In [71]:
# TypeT5 does best on everything it predicts on, so simply take all of it
adjusted_fill_me_in = pd.merge(
    left=adjusted_fill_me_in,
    right=typet5_test_adjusted[key + ["anno"]].rename(columns={"anno": "typet5_adjusted_anno"}),
    on=key,
    how="left"
)

# Fill in remainder with Type4Py
adjusted_fill_me_in = pd.merge(
    left=adjusted_fill_me_in,
    right=type4py_test_adjusted[key + ["anno"]].rename(columns={"anno": "type4py_adjusted_anno"}),
    on=key,
    how="left"
)

adjusted_fill_me_in = pd.merge(
    left=adjusted_fill_me_in,
    right=typilus_test_adjusted[key + ["anno"]].rename(columns={"anno": "typilus_adjusted_anno"}),
    on=key,
    how="left"
)

In [72]:
selected_adjusted = adjusted_fill_me_in.assign(
    anno=adjusted_fill_me_in.typet5_adjusted_anno \
        .fillna(adjusted_fill_me_in.type4py_adjusted_anno) \
        .fillna(adjusted_fill_me_in.typilus_adjusted_anno)
).drop(columns=[
    'typet5_adjusted_anno', 
    'type4py_adjusted_anno', 
    'typilus_adjusted_anno'
])

joined_adjusted = experiments.inferred.join_truth_to_preds(
    truth=test_gt,
    predictions=selected_adjusted,
    comparable_anno="adjusted_anno",
)

adjusted_eval = experiments.inferred.evaluatable(joined_adjusted).replace("<MISSING>", pd.NA)

/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


In [73]:
display(adjusted_selected_perf := experiments.predictions.performance(
    adjusted_eval,
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18871,18165,706,15473,0.819935,0.851803
common,10583,10174,409,6544,0.618350,0.643208
rare,12108,11486,622,6432,0.531219,0.559986
total,41562,39825,1737,28449,0.684495,0.714350


In [74]:
base_fill_me_in = test_gt.drop(columns=[
    'raw_anno',
    'depth_limited_anno',
    'adjusted_anno',
    'base_anno',
    'context_category',
    'nested'
])
display(base_fill_me_in.columns)
display(base_fill_me_in.shape)

key = ["repository", "file", "category", "qname_ssa", "qname"]

Index(['file', 'category', 'qname', 'qname_ssa', 'repository'], dtype='object')

(229080, 5)

In [75]:
# TypeT5 does best on everything it predicts on, so simply take all of it
base_fill_me_in = pd.merge(
    left=base_fill_me_in,
    right=typet5_test_base[key + ["anno"]].rename(columns={"anno": "typet5_base_anno"}),
    on=key,
    how="left"
)

# Fill in remainder with Type4Py
base_fill_me_in = pd.merge(
    left=base_fill_me_in,
    right=type4py_test_base[key + ["anno"]].rename(columns={"anno": "type4py_base_anno"}),
    on=key,
    how="left"
)

base_fill_me_in = pd.merge(
    left=base_fill_me_in,
    right=typilus_test_base[key + ["anno"]].rename(columns={"anno": "typilus_base_anno"}),
    on=key,
    how="left"
)

In [76]:
selected_base = base_fill_me_in.assign(
    anno=base_fill_me_in.typet5_base_anno \
        .fillna(base_fill_me_in.type4py_base_anno) \
        .fillna(base_fill_me_in.typilus_base_anno)
).drop(columns=[
    'typet5_base_anno', 
    'type4py_base_anno', 
    'typilus_base_anno'
])

joined_base = experiments.inferred.join_truth_to_preds(
    truth=test_gt,
    predictions=selected_base,
    comparable_anno="base_anno",
)

base_eval = experiments.inferred.evaluatable(joined_base).replace("<MISSING>", pd.NA)

/nfs/home/bsparks/mdti4py/scripts/experiments/inferred.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned[clean_annos] = cleaned[clean_annos].fillna("<MISSING>")


In [77]:
display(base_selected_perf := experiments.predictions.performance(
    base_eval,
    ubiq_types=ubiq_types,
    comm_types=common_types,
    rare_types=rare_types,
    total=True
))

,observations,predictions,unassigned,matches,stracc,relacc
ubiquitous,18871,18165,706,15773,0.835833,0.868318
common,10583,10174,409,7655,0.723330,0.752408
rare,12108,11486,622,7419,0.612735,0.645917
total,41562,39825,1737,30847,0.742192,0.774564


In [96]:
display(selected_perf := pd.concat(
    [
        adjusted_selected_perf[["observations", "predictions"]],
        adjusted_selected_perf[["stracc", "relacc"]],
        base_selected_perf[["stracc", "relacc"]],
    ],
    keys=["", "adjusted", "base"],
    axis=1,
))

display(tt5_test_perf := pd.concat(
    [
        typet5_test_adj_perf[["observations", "predictions"]],
        typet5_test_adj_perf[["stracc", "relacc"]],
        typet5_test_base_perf[["stracc", "relacc"]],
    ],
    keys=["", "adjusted", "base"],
    axis=1,
))

adjusted                base          
           observations predictions    stracc    relacc    stracc    relacc
ubiquitous        18871       18165  0.819935  0.851803  0.835833  0.868318
common            10583       10174  0.618350  0.643208  0.723330  0.752408
rare              12108       11486  0.531219  0.559986  0.612735  0.645917
total             41562       39825  0.684495  0.714350  0.742192  0.774564

adjusted                base          
           observations predictions    stracc    relacc    stracc    relacc
ubiquitous        17903       15045  0.752332  0.895248  0.766352  0.911931
common             9623        8385  0.630469  0.723554  0.724099  0.831008
rare              10887        9472  0.574906  0.660790  0.648572  0.745460
total             38413       32902  0.671517  0.783995  0.722386  0.843383

In [97]:
display(by_model_perf := pd.concat(
    [tt5_test_perf, selected_perf],
    keys=["TypeT5", "Select(TT5,T4P,Typ)"]
))

by_model_perf.index = by_model_perf.index.set_names(["model", "scarcity"])

by_model_perf = pd.concat([g for _, g in by_model_perf.swaplevel().groupby("scarcity", sort=False)])
display(by_model_perf)

adjusted            \
                               observations predictions    stracc    relacc   
TypeT5              ubiquitous        17903       15045  0.752332  0.895248   
                    common             9623        8385  0.630469  0.723554   
                    rare              10887        9472  0.574906  0.660790   
                    total             38413       32902  0.671517  0.783995   
Select(TT5,T4P,Typ) ubiquitous        18871       18165  0.819935  0.851803   
                    common            10583       10174  0.618350  0.643208   
                    rare              12108       11486  0.531219  0.559986   
                    total             41562       39825  0.684495  0.714350   

                                    base            
                                  stracc    relacc  
TypeT5              ubiquitous  0.766352  0.911931  
                    common      0.724099  0.831008  
                    rare        0.648572  0.745460  
                    total       0.722386  0.843383  
Select(TT5,T4P,Typ) ubiquitous  0.835833  0.868318  
                    common      0.723330  0.752408  
                    rare        0.612735  0.645917  
                    total       0.742192  0.774564

adjusted            \
                               observations predictions    stracc    relacc   
scarcity   model                                                              
ubiquitous TypeT5                     17903       15045  0.752332  0.895248   
           Select(TT5,T4P,Typ)        18871       18165  0.819935  0.851803   
common     TypeT5                      9623        8385  0.630469  0.723554   
           Select(TT5,T4P,Typ)        10583       10174  0.618350  0.643208   
rare       TypeT5                     10887        9472  0.574906  0.660790   
           Select(TT5,T4P,Typ)        12108       11486  0.531219  0.559986   
total      TypeT5                     38413       32902  0.671517  0.783995   
           Select(TT5,T4P,Typ)        41562       39825  0.684495  0.714350   

                                    base            
                                  stracc    relacc  
scarcity   model                                    
ubiquitous TypeT5               0.766352  0.911931  
           Select(TT5,T4P,Typ)  0.835833  0.868318  
common     TypeT5               0.724099  0.831008  
           Select(TT5,T4P,Typ)  0.723330  0.752408  
rare       TypeT5               0.648572  0.745460  
           Select(TT5,T4P,Typ)  0.612735  0.645917  
total      TypeT5               0.722386  0.843383  
           Select(TT5,T4P,Typ)  0.742192  0.774564

In [98]:
print(by_model_perf.to_latex(
    float_format="{:.2f}".format,
))

\begin{tabular}{llrrrrrr}
\toprule
 &  & \multicolumn{2}{r}{} & \multicolumn{2}{r}{adjusted} & \multicolumn{2}{r}{base} \\
 &  & observations & predictions & stracc & relacc & stracc & relacc \\
scarcity & model &  &  &  &  &  &  \\
\midrule
\multirow[t]{2}{*}{ubiquitous} & TypeT5 & 17903 & 15045 & 0.75 & 0.90 & 0.77 & 0.91 \\
 & Select(TT5,T4P,Typ) & 18871 & 18165 & 0.82 & 0.85 & 0.84 & 0.87 \\
\cline{1-8}
\multirow[t]{2}{*}{common} & TypeT5 & 9623 & 8385 & 0.63 & 0.72 & 0.72 & 0.83 \\
 & Select(TT5,T4P,Typ) & 10583 & 10174 & 0.62 & 0.64 & 0.72 & 0.75 \\
\cline{1-8}
\multirow[t]{2}{*}{rare} & TypeT5 & 10887 & 9472 & 0.57 & 0.66 & 0.65 & 0.75 \\
 & Select(TT5,T4P,Typ) & 12108 & 11486 & 0.53 & 0.56 & 0.61 & 0.65 \\
\cline{1-8}
\multirow[t]{2}{*}{total} & TypeT5 & 38413 & 32902 & 0.67 & 0.78 & 0.72 & 0.84 \\
 & Select(TT5,T4P,Typ) & 41562 & 39825 & 0.68 & 0.71 & 0.74 & 0.77 \\
\cline{1-8}
\bottomrule
\end{tabular}

